In [1]:
# MARK:- libs
import pandas as pd 
import numpy as np 
from datetime import datetime, date, timedelta

In [2]:
# MARK:- read input
path = 'reservation.csv'
data_df = pd.read_csv(path)
data_df.check_in_date = pd.to_datetime(data_df.check_in_date)
data_df.check_out_date = pd.to_datetime(data_df.check_out_date)

In [3]:
# MARK:- calculating number of nights and price for each night for each period
data_df['night_no'] = data_df.apply(lambda row: (row['check_out_date'] - row['check_in_date']).days, axis=1)
data_df['punit'] = data_df.price / data_df.night_no
data_df

,check_in_date,check_out_date,price,night_no,punit
0,2018-10-14,2018-10-15,226495,1,226495.0
1,2018-10-15,2018-10-16,227950,1,227950.0
2,2018-10-20,2018-10-21,392850,1,392850.0
3,2018-10-25,2018-10-28,1076700,3,358900.0
4,2018-10-25,2018-10-26,358900,1,358900.0
...,...,...,...,...,...
317,2019-08-31,2019-09-01,682000,1,682000.0
318,2019-09-01,2019-09-02,426800,1,426800.0
319,2019-10-11,2019-10-12,426800,1,426800.0
320,2019-10-25,2019-10-26,545600,1,545600.0


In [4]:
# MARK:- requirements for function f
min_range = min(data_df.check_in_date).date()
max_range = max(data_df.check_out_date).date()
relieve = pd.DataFrame(pd.date_range(min_range, max_range))
relieve.columns = ['day']
relieve['prices'] = relieve.day.apply(lambda x: [row['punit'] for idx, row in data_df.iterrows() if row['check_in_date'] <= x < row['check_out_date']])
relieve['maxprice'] = relieve.prices.apply(lambda x : 0 if not x else max(x))

In [5]:
def f(x):
    if x not in relieve.day.astype(str).values:
        return
    return relieve[relieve.day.astype(str) == x].maxprice.values[0]
f('2018-11-17')

407400.0

In [6]:
max_sum = relieve.maxprice.sum()
max_sum

133731735.0